
# 00_baixar_dados — Ingestão CCCAGG (CryptoCompare/CCData) · **MVP**

**Objetivo:** baixar/atualizar histórico **BTC/USD 1h** (CCCAGG), com:
- retomada automática (append a partir do último `open_time`),
- ou backfill de uma janela (`since`/`until`),
- paginação com *retry/backoff*,
- escrita **atômica** (arquivo temporário + rename),
- **deduplicação** e **ordenação** por `open_time`,
- **validações mínimas** (gaps/monotonicidade/valores),
- `meta.json` com métricas da execução.

> ⚠️ Este notebook cria funções e **não** dispara o download automaticamente.  
> Execute a célula **“Run (opcional)”** somente no seu ambiente com internet e `CRYPTOCOMPARE_API_KEY` (opcional).


## Imports e configuração

In [ ]:

import os, time, math, json, gc
import requests
import pandas as pd
from pathlib import Path
from datetime import datetime, timezone
from typing import Optional, Any, Dict

# Parâmetros padrão (MVP)
DEFAULT_FSYM = "BTC"
DEFAULT_TSYM = "USD"
DEFAULT_TF   = "1h"  # fixo neste MVP
DEFAULT_OUT  = "../data/raw/BTCUSD_CCCAGG_1h.csv"
DEFAULT_BATCH = 2000
DEFAULT_SLEEP = 0.12  # respeitar rate limit leve
API_KEY = os.getenv("CRYPTOCOMPARE_API_KEY")  # opcional
BASE_URL = "https://min-api.cryptocompare.com/data/v2/histohour"  # 1h


## Helpers (tempo, I/O, validações)

In [ ]:

def _to_epoch_seconds(x: Optional[Any]) -> Optional[int]:
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return int(x)
    if isinstance(x, str):
        x = x.strip()
        try:
            if len(x) <= 10:
                dt = datetime.strptime(x, "%Y-%m-%d").replace(tzinfo=timezone.utc)
            elif len(x) <= 16:
                dt = datetime.strptime(x, "%Y-%m-%d %H:%M").replace(tzinfo=timezone.utc)
            else:
                dt = datetime.fromisoformat(x.replace("Z","")).astimezone(timezone.utc)
            return int(dt.timestamp())
        except Exception:
            try:
                return int(float(x))
            except Exception:
                raise ValueError(f"Não foi possível interpretar 'since/until': {x}")
    raise TypeError(f"Tipo não suportado para timestamp: {type(x)}")

def _read_last_ms(csv_path: Path) -> Optional[int]:
    if not csv_path.exists():
        return None
    try:
        s = pd.read_csv(csv_path, usecols=["open_time"]).open_time
        if len(s) == 0:
            return None
        return int(s.max())
    except Exception:
        return None

def _headers() -> Dict[str, str]:
    return {"authorization": f"Apikey {API_KEY}"} if API_KEY else {}

def _sleep_backoff(attempt: int, base: float = 0.5, cap: float = 8.0):
    t = min(cap, base * (2 ** attempt))
    time.sleep(t)

def _validate_ohlcv(df: pd.DataFrame, verbose=True):
    msgs = []
    ok = True
    if not df.index.is_monotonic_increasing:
        ok = False; msgs.append("Index não está ordenado ASC.")
    for col in ["open","high","low","close"]:
        if (df[col] <= 0).any():
            ok = False; msgs.append(f"Valores <=0 em {col}.")
    if ((df["high"] < df[["open","close"]].max(axis=1)) | (df["low"] > df[["open","close"]].min(axis=1))).any():
        msgs.append("Aviso: 'high/low' fora do envelope em algumas linhas.")
    diffs = (df.index[1:] - df.index[:-1]).total_seconds()
    if len(diffs) > 0 and (diffs > 5400).any():
        msgs.append("Aviso: gaps > 90min detectados.")
    if verbose:
        if ok: print("Validações básicas OK.")
        for m in msgs: print("•", m)
    return ok, msgs


## Cliente: requisição com retry/backoff (histohour)

In [ ]:

def _fetch_histohour_page(fsym: str, tsym: str, limit: int, toTs: Optional[int]) -> list:
    params = dict(fsym=fsym, tsym=tsym, limit=limit, e="CCCAGG")
    if toTs is not None:
        params["toTs"] = int(toTs)
    max_attempts = 6
    for attempt in range(max_attempts):
        try:
            r = requests.get(BASE_URL, params=params, headers=_headers(), timeout=20)
            if r.status_code in (429, 500, 502, 503, 504):
                raise requests.HTTPError(f"HTTP {r.status_code}")
            r.raise_for_status()
            data = r.json().get("Data", {}).get("Data", [])
            return data or []
        except Exception as e:
            if attempt == max_attempts - 1:
                raise
            _sleep_backoff(attempt)
    return []


## Função principal: download + merge/dedup/sort + meta

In [ ]:
def download_cccagg_ohlcv_csv(
    fsym=DEFAULT_FSYM, tsym=DEFAULT_TSYM, out_csv=DEFAULT_OUT,
    since=None, until=None, batch=DEFAULT_BATCH, sleep=DEFAULT_SLEEP, verbose=True,
    checkpoint_every_page=True  # <- novo
):
    out_path = Path(out_csv)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    stage_path = out_path.with_suffix(".staging.csv")  # <- arquivo de staging

    start_ts = _to_epoch_seconds(since)
    end_ts   = _to_epoch_seconds(until)

    # ler último ms do final e do staging
    last_final_ms  = _read_last_ms(out_path) or None
    last_stage_ms  = _read_last_ms(stage_path) or None

    # Se não foi passado since/until, retoma do maior dos arquivos
    if start_ts is None and end_ts is None:
        last_any = max([x for x in [last_final_ms, last_stage_ms] if x is not None], default=None)
        if last_any:
            start_ts = int(last_any/1000) + 3600
            if verbose:
                print("Retomando de", pd.to_datetime(start_ts, unit="s", utc=True))

    cursor = end_ts
    total_pages = 0
    total_rows_written_stage = 0

    # escrever header no staging se ele não existir
    if checkpoint_every_page and not stage_path.exists():
        stage_path.parent.mkdir(parents=True, exist_ok=True)
        with open(stage_path, "w", encoding="utf-8") as f:
            f.write("open_time,open,high,low,close,volume,notional_usd\n")

    while True:
        page = _fetch_histohour_page(fsym, tsym, batch, cursor)
        total_pages += 1
        if not page:
            if verbose: print("Página vazia; encerrando.")
            break

        page.sort(key=lambda d: d["time"])
        # filtra por janela e mapeia
        rows = []
        for d in page:
            t = d["time"]
            if start_ts is not None and t < start_ts:
                continue
            if end_ts is not None and t > end_ts:
                continue
            rows.append([t*1000, d["open"], d["high"], d["low"], d["close"], d["volumefrom"], d["volumeto"]])

        if rows and checkpoint_every_page:
            # append incremental no staging (pode ficar fora de ordem; ordenaremos no merge final)
            with open(stage_path, "a", encoding="utf-8") as f:
                for r2 in rows:
                    f.write(",".join(str(x) for x in r2) + "\n")
                f.flush()
                os.fsync(f.fileno())
            total_rows_written_stage += len(rows)

        if verbose:
            ts_ult = page[-1]["time"]
            print(f"+{len(page)} (até {pd.to_datetime(ts_ult, unit='s', utc=True)})")

        cursor = page[0]["time"] - 1
        if start_ts is not None and cursor < start_ts:
            break
        time.sleep(max(0.0, sleep))

    # Se não fez checkpoint por página, ainda precisamos gravar algo (comportamento antigo)
    if not checkpoint_every_page:
        if verbose: print("Checkpoint por página desativado — nada salvo porque o processo foi interrompido ou não havia novas linhas.")
        return out_path

    # Merge staging + final → final (dedup + sort)
    # Obs.: mesmo se a execução for interrompida antes desta etapa, o progresso já está em stage_path.
    if stage_path.exists():
        df_stage = pd.read_csv(stage_path)
        df_stage = df_stage.drop_duplicates("open_time")

        if out_path.exists():
            df_final = pd.read_csv(out_path)
            df = pd.concat([df_final, df_stage], ignore_index=True)
        else:
            df = df_stage

        df = df.drop_duplicates("open_time", keep="last").sort_values("open_time").reset_index(drop=True)

        # escrita atômica do final
        tmp_out = out_path.with_suffix(".csv.tmp")
        df.to_csv(tmp_out, index=False)
        tmp_out.replace(out_path)

        # staging pode ser removido (ou mantido, se quiser staged commits)
        try:
            stage_path.unlink()
        except Exception:
            pass

        # meta
        meta = {
            "run_id": datetime.utcnow().strftime("%Y%m%d_%H%M%S") + "_1h",
            "fsym": fsym, "tsym": tsym, "tf": "1h", "exchange": "CCCAGG",
            "rows_final": int(len(df)),
            "rows_added_this_run": int(len(df_stage)),
            "start_utc": pd.to_datetime(df.open_time.iloc[0], unit="ms", utc=True).isoformat(),
            "end_utc":   pd.to_datetime(df.open_time.iloc[-1], unit="ms", utc=True).isoformat(),
            "pages": int(total_pages),
            "checkpointed_rows": int(total_rows_written_stage),
            "batch": batch
        }
        with open(out_path.with_suffix(".meta.json"), "w", encoding="utf-8") as f:
            json.dump(meta, f, indent=2)

        if verbose:
            print(f"Merge concluído. Linhas totais: {len(df)} | Adicionadas: {len(df_stage)}")
            print("Arquivo final:", str(out_path))
    else:
        if verbose:
            print("Nenhum dado em staging para consolidar.")

    return out_path

## Validações rápidas (após download)

In [ ]:

def post_validate_csv(out_csv: str):
    path = Path(out_csv)
    if not path.exists():
        print("Arquivo não encontrado:", path); return False
    df = pd.read_csv(path)
    dt = pd.to_datetime(df["open_time"], unit="ms", utc=True)
    df_idx = df.set_index(dt.tz_convert(None)).drop(columns=["open_time"]).sort_index()
    ok, msgs = _validate_ohlcv(df_idx, verbose=True)
    return ok

# Exemplo (rodar só após o download):
# post_validate_csv(DEFAULT_OUT)


## Run (opcional) — execute no seu ambiente com internet

In [ ]:

# ⚠️ NÃO EXECUTE AQUI se seu ambiente não tiver internet.
# Exemplo 1: retomar automaticamente
download_cccagg_ohlcv_csv(out_csv=DEFAULT_OUT, verbose=True)

# Exemplo 2: backfill exato de 2020-01-01 a 2022-12-31
# download_cccagg_ohlcv_csv(since="2020-01-01", until="2022-12-31", out_csv=DEFAULT_OUT, verbose=True)

# Depois, checar:
# post_validate_csv(DEFAULT_OUT)
